# Governance PID-Lag Toy Model Explorer

This notebook provides an interactive environment for exploring the Governance PID-Lag Toy Model. It allows you to visualize how different parameter settings affect governance responses to technological acceleration and jolts.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display

# Add parent directory to path to import our modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import our modules
from controller import PIDParams, LagParams, PIDLagController
from technology import TechProgressParams, simulate_tech_progress

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12

## Basic Simulation

First, let's run a basic simulation with default parameters to understand the model's behavior.

In [ ]:
# Default parameters
tech_params = TechProgressParams(
    base_growth_rate=0.05,
    jolt_time=50,
    jolt_magnitude=0.2,
    jolt_duration=5.0,
    jolt_decay=0.5,
    noise_std=0.01,
    initial_value=1.0
)

pid_params = PIDParams(
    kp=0.5,
    ki=0.1,
    kd=0.2,
    setpoint=0.2,
    windup_guard=20.0,
    sample_time=1.0
)

lag_params = LagParams(
    response_delay=5,
    response_time=10,
    decay_rate=0.1,
    effectiveness_cap=0.8
)

# Create controller
controller = PIDLagController(pid_params, lag_params)

# Run simulation without governance
results_no_gov = simulate_tech_progress(
    tech_params,
    governance_func=None,
    time_range=(0, 100),
    time_step=1.0,
    risk_threshold=10.0,
    risk_steepness=2.0,
    seed=42
)

# Create governance function
def governance_func(t, tech_level):
    # Calculate risk
    risk = 1 / (1 + np.exp(-2.0 * (tech_level - 10.0)))
    # Update controller with risk as process variable
    return controller.update(risk, t)

# Run simulation with governance
results_with_gov = simulate_tech_progress(
    tech_params,
    governance_func=governance_func,
    time_range=(0, 100),
    time_step=1.0,
    risk_threshold=10.0,
    risk_steepness=2.0,
    seed=42
)

In [ ]:
# Create visualization
fig, axs = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Plot technology level
axs[0].plot(results_no_gov['time'], results_no_gov['tech_level'], 
            label='Without Governance', color='blue', linewidth=2)
axs[0].plot(results_with_gov['time'], results_with_gov['tech_level'], 
            label='With Governance', color='green', linewidth=2)

# Add jolt marker
axs[0].axvline(x=tech_params.jolt_time, color='red', linestyle='--', 
              label='Jolt Occurs', alpha=0.7)

axs[0].set_ylabel('Technology Level')
axs[0].set_title('Technology Progress with and without Governance')
axs[0].legend()
axs[0].grid(True, alpha=0.3)

# Plot risk
axs[1].plot(results_no_gov['time'], results_no_gov['risk'], 
            label='Without Governance', color='blue', linewidth=2)
axs[1].plot(results_with_gov['time'], results_with_gov['risk'], 
            label='With Governance', color='green', linewidth=2)
axs[1].axhline(y=pid_params.setpoint, color='black', linestyle='--', 
              label='Risk Setpoint', alpha=0.7)
axs[1].axvline(x=tech_params.jolt_time, color='red', linestyle='--', alpha=0.7)

axs[1].set_ylabel('Risk Level')
axs[1].set_title('Risk Level with and without Governance')
axs[1].legend()
axs[1].grid(True, alpha=0.3)

# Plot governance effect
axs[2].plot(results_with_gov['time'], results_with_gov['governance_effect'], 
            label='Governance Effect', color='purple', linewidth=2)
axs[2].plot(results_with_gov['time'], results_with_gov['jolt_effect'], 
            label='Jolt Effect', color='red', linewidth=2)
axs[2].axvline(x=tech_params.jolt_time, color='red', linestyle='--', alpha=0.7)

axs[2].set_xlabel('Time')
axs[2].set_ylabel('Effect Magnitude')
axs[2].set_title('Governance and Jolt Effects')
axs[2].legend()
axs[2].grid(True, alpha=0.3)

# Add overall title
plt.suptitle(f'Governance PID-Lag Model Simulation\nKp={pid_params.kp}, Ki={pid_params.ki}, Kd={pid_params.kd}, Delay={lag_params.response_delay}, Response Time={lag_params.response_time}', 
            fontsize=16)

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

## Interactive Parameter Exploration

Now let's create an interactive widget to explore how different parameter settings affect the model's behavior.

In [ ]:
def run_interactive_simulation(base_growth_rate=0.05, jolt_time=50, jolt_magnitude=0.2,
                              kp=0.5, ki=0.1, kd=0.2, setpoint=0.2,
                              response_delay=5, response_time=10, effectiveness_cap=0.8):
    """
    Run simulation with interactive parameters and plot results.
    """
    # Create parameters
    tech_params = TechProgressParams(
        base_growth_rate=base_growth_rate,
        jolt_time=jolt_time if jolt_time >= 0 else None,
        jolt_magnitude=jolt_magnitude,
        jolt_duration=5.0,
        jolt_decay=0.5,
        noise_std=0.01,
        initial_value=1.0
    )
    
    pid_params = PIDParams(
        kp=kp,
        ki=ki,
        kd=kd,
        setpoint=setpoint,
        windup_guard=20.0,
        sample_time=1.0
    )
    
    lag_params = LagParams(
        response_delay=response_delay,
        response_time=response_time,
        decay_rate=0.1,
        effectiveness_cap=effectiveness_cap
    )
    
    # Create controller
    controller = PIDLagController(pid_params, lag_params)
    
    # Run simulation without governance
    results_no_gov = simulate_tech_progress(
        tech_params,
        governance_func=None,
        time_range=(0, 100),
        time_step=1.0,
        risk_threshold=10.0,
        risk_steepness=2.0,
        seed=42
    )
    
    # Create governance function
    def governance_func(t, tech_level):
        # Calculate risk
        risk = 1 / (1 + np.exp(-2.0 * (tech_level - 10.0)))
        # Update controller with risk as process variable
        return controller.update(risk, t)
    
    # Run simulation with governance
    results_with_gov = simulate_tech_progress(
        tech_params,
        governance_func=governance_func,
        time_range=(0, 100),
        time_step=1.0,
        risk_threshold=10.0,
        risk_steepness=2.0,
        seed=42
    )
    
    # Create visualization
    fig, axs = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
    
    # Plot technology level
    axs[0].plot(results_no_gov['time'], results_no_gov['tech_level'], 
                label='Without Governance', color='blue', linewidth=2)
    axs[0].plot(results_with_gov['time'], results_with_gov['tech_level'], 
                label='With Governance', color='green', linewidth=2)
    
    # Add jolt marker if applicable
    if jolt_time >= 0:
        axs[0].axvline(x=jolt_time, color='red', linestyle='--', 
                      label='Jolt Occurs', alpha=0.7)
    
    axs[0].set_ylabel('Technology Level')
    axs[0].set_title('Technology Progress with and without Governance')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
    
    # Plot risk
    axs[1].plot(results_no_gov['time'], results_no_gov['risk'], 
                label='Without Governance', color='blue', linewidth=2)
    axs[1].plot(results_with_gov['time'], results_with_gov['risk'], 
                label='With Governance', color='green', linewidth=2)
    axs[1].axhline(y=setpoint, color='black', linestyle='--', 
                  label='Risk Setpoint', alpha=0.7)
    
    if jolt_time >= 0:
        axs[1].axvline(x=jolt_time, color='red', linestyle='--', alpha=0.7)
    
    axs[1].set_ylabel('Risk Level')
    axs[1].set_title('Risk Level with and without Governance')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)
    
    # Plot governance effect
    axs[2].plot(results_with_gov['time'], results_with_gov['governance_effect'], 
                label='Governance Effect', color='purple', linewidth=2)
    axs[2].plot(results_with_gov['time'], results_with_gov['jolt_effect'], 
                label='Jolt Effect', color='red', linewidth=2)
    
    if jolt_time >= 0:
        axs[2].axvline(x=jolt_time, color='red', linestyle='--', alpha=0.7)
    
    axs[2].set_xlabel('Time')
    axs[2].set_ylabel('Effect Magnitude')
    axs[2].set_title('Governance and Jolt Effects')
    axs[2].legend()
    axs[2].grid(True, alpha=0.3)
    
    # Add overall title
    plt.suptitle(f'Governance PID-Lag Model Simulation\nKp={kp}, Ki={ki}, Kd={kd}, Delay={response_delay}, Response Time={response_time}', 
                fontsize=16)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    
    # Calculate key metrics
    max_tech_no_gov = np.max(results_no_gov['tech_level'])
    max_tech_with_gov = np.max(results_with_gov['tech_level'])
    max_risk_no_gov = np.max(results_no_gov['risk'])
    max_risk_with_gov = np.max(results_with_gov['risk'])
    avg_risk_no_gov = np.mean(results_no_gov['risk'])
    avg_risk_with_gov = np.mean(results_with_gov['risk'])
    
    # Find time when risk exceeds 0.5
    risk_threshold_time_no_gov = None
    for i, risk in enumerate(results_no_gov['risk']):
        if risk > 0.5:
            risk_threshold_time_no_gov = results_no_gov['time'][i]
            break
    
    risk_threshold_time_with_gov = None
    for i, risk in enumerate(results_with_gov['risk']):
        if risk > 0.5:
            risk_threshold_time_with_gov = results_with_gov['time'][i]
            break
    
    # Calculate time delay in response to jolt
    if jolt_time >= 0:
        # Find peak governance response after jolt
        jolt_idx = np.argmin(np.abs(results_with_gov['time'] - jolt_time))
        gov_response = results_with_gov['governance_effect'][jolt_idx:]
        peak_response_idx = np.argmax(gov_response) + jolt_idx
        peak_response_time = results_with_gov['time'][peak_response_idx]
        response_delay_actual = peak_response_time - jolt_time
    else:
        response_delay_actual = "N/A"
    
    # Print metrics
    print("\nKey Metrics:")
    print(f"  Max Tech Level: {max_tech_no_gov:.2f} (No Gov) vs {max_tech_with_gov:.2f} (With Gov)")
    print(f"  Max Risk Level: {max_risk_no_gov:.2f} (No Gov) vs {max_risk_with_gov:.2f} (With Gov)")
    print(f"  Avg Risk Level: {avg_risk_no_gov:.2f} (No Gov) vs {avg_risk_with_gov:.2f} (With Gov)")
    print(f"  Time to Risk > 0.5: {risk_threshold_time_no_gov} (No Gov) vs {risk_threshold_time_with_gov} (With Gov)")
    print(f"  Jolt Response Delay: {response_delay_actual}")

In [ ]:
# Create interactive widget
interact(
    run_interactive_simulation,
    base_growth_rate=widgets.FloatSlider(min=0.01, max=0.1, step=0.01, value=0.05, description='Growth Rate:'),
    jolt_time=widgets.IntSlider(min=-1, max=80, step=5, value=50, description='Jolt Time:'),
    jolt_magnitude=widgets.FloatSlider(min=0.0, max=0.5, step=0.05, value=0.2, description='Jolt Magnitude:'),
    kp=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description='Kp:'),
    ki=widgets.FloatSlider(min=0.0, max=0.5, step=0.05, value=0.1, description='Ki:'),
    kd=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.2, description='Kd:'),
    setpoint=widgets.FloatSlider(min=0.1, max=0.5, step=0.05, value=0.2, description='Risk Setpoint:'),
    response_delay=widgets.IntSlider(min=0, max=20, step=1, value=5, description='Response Delay:'),
    response_time=widgets.IntSlider(min=1, max=20, step=1, value=10, description='Response Time:'),
    effectiveness_cap=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.8, description='Effectiveness Cap:')
)

## Conclusion

This notebook demonstrates the Governance PID-Lag Toy Model for simulating governance responses to technological acceleration and jolts. Key insights include:

1. **Response Delay Impact**: Longer response delays significantly reduce governance effectiveness, especially for rapid jolts
2. **Effectiveness Cap Limitations**: Even with perfect detection and response timing, limited governance effectiveness can result in uncontrolled risk
3. **PID Tuning Importance**: Proper tuning of PID parameters is crucial for effective governance response
4. **Jolt Magnitude Challenge**: Larger jolts are more difficult to control, potentially overwhelming governance mechanisms

These insights have important implications for real-world governance of emerging technologies, suggesting the need for proactive monitoring, rapid response capabilities, and robust governance mechanisms.